In [127]:
import pandas as pd

Load Data

In [73]:
nhanesII = pd.read_stata("/Users/bryankim/Documents - Local/NBER/Case Deaton/Data/FRS/NHANESII/nhanesII_frs.dta")
nhanesII.head()
test=nhanesII.head()

See Ranges

In [134]:
print(nhanesII[["AGEYR_AT_INT","SEX","lbdhdd","lbxtc","treated","asbp","curr_smq"]].min())
print(nhanesII[["AGEYR_AT_INT","SEX","lbdhdd","lbxtc","treated","asbp","curr_smq"]].max())
nhanesII.curr_smq.value_counts()
nhanesII["curr_smq"][1]==1

AGEYR_AT_INT    45.0
SEX              1.0
lbdhdd          15.0
lbxtc           80.0
treated          0.0
asbp            70.0
curr_smq         0.0
dtype: float64
AGEYR_AT_INT     74.0
SEX               2.0
lbdhdd          187.0
lbxtc           644.0
treated           1.0
asbp            300.0
curr_smq          1.0
dtype: float64


False

Create `treated` variable

Define `check` functions for females

In [136]:
def ftchol_pts(totchol,age):
    if totchol < 160:
        return(0)
    if totchol in range(160,199+1):
        if age in range(20,39+1): return(4)
        if age in range(40,49+1): return(3) 
        if age in range(50,59+1): return(2) 
        if age in range(60,69+1): return(1) 
        if age >= 70: return(1) 
    if totchol in range(200,239+1):
        if age in range(20,39+1): return(8) 
        if age in range(40,49+1): return(6) 
        if age in range(50,59+1): return(4) 
        if age in range(60,69+1): return(2) 
        if age >= 70: return(1) 
    if totchol in range(240,279+1):
        if age in range(20,39+1): return(11) 
        if age in range(40,49+1): return(8) 
        if age in range(50,59+1): return(5) 
        if age in range(60,69+1): return(3) 
        if age >= 70: return(2) 
    if totchol >= 280:
        if age in range(20,39+1): return(13) 
        if age in range(40,49+1): return(10)
        if age in range(50,59+1): return(7)
        if age in range(60,69+1): return(4) 
        if age >= 70: return(2) 
    
def fsmoke_pts(curr_smq,age):
    if curr_smq == 0:
        return(0)
    if curr_smq == 1:
        if age in range(20,39+1): return(9) 
        if age in range(40,49+1): return(7) 
        if age in range(50,59+1): return(4) 
        if age in range(60,69+1): return(2)
        if age >= 70: return(1) 

    
def fsbp_pts(sbp,treated):
    if sbp < 120:
        return(0)
    if sbp in range(120,129+1):
        return(3) if treated == 1 else 1
    if sbp in range(130,139+1):
        return(4) if treated == 1 else 2
    if sbp in range(140,159+1):
        return(5) if treated == 1 else 3
    if sbp >= 160:
        return(6) if treated == 1 else 4
    
def fage_pts(age):
    if age in range(45,49+1):
        return(3)
    if age in range(50,54+1):
        return(6)
    if age in range(55,59+1):
        return(8)
    if age in range(60,64+1):
        return(10)
    if age in range(65,69+1):
        return(12)
    if age in range(70,74+1):
        return(14)
    if age >= 75:
        return(16)

def fhdl_pts(hdl):
    if hdl >= 60:
        return(-1)
    if hdl in range(50,59+1):
        return(0)
    if hdl in range(40,49+1):
        return(1)
    if hdl < 40:
        return(2)

Define `check` functions for males

In [147]:
def mtchol_pts(totchol,age):
    if totchol < 160:
        return(0)
    if totchol in range(160,199+1):
        if age in range(20,39+1): return(4)
        if age in range(40,49+1): return(3) 
        if age in range(50,59+1): return(2) 
        if age in range(60,69+1): return(1) 
        if age >= 70: return(0) 
    if totchol in range(200,239+1):
        if age in range(20,39+1): return(7) 
        if age in range(40,49+1): return(5) 
        if age in range(50,59+1): return(3) 
        if age in range(60,69+1): return(1) 
        if age >= 70: return(0) 
    if totchol in range(240,279+1):
        if age in range(20,39+1): return(9) 
        if age in range(40,49+1): return(6) 
        if age in range(50,59+1): return(4) 
        if age in range(60,69+1): return(2) 
        if age >= 70: return(1) 
    if totchol >= 280:
        if age in range(20,39+1): return(11) 
        if age in range(40,49+1): return(8)
        if age in range(50,59+1): return(5)
        if age in range(60,69+1): return(3) 
        if age >= 70:return(1) 


#
 
def msmoke_pts(curr_smq,age):
    if curr_smq == 0:
        return(0)
    if curr_smq == 1:
        if age in range(20,39+1): return(8) 
        if age in range(40,49+1): return(5) 
        if age in range(50,59+1): return(3) 
        if age in range(60,69+1): return(1)
        if age >= 70: return(1)
    
def msbp_pts(sbp,treated):
    if sbp < 120:
        return(0)
    if sbp in range(120,129+1):
        return(1) if treated == 1 else 0
    if sbp in range(130,159+1):
        return(2) if treated == 1 else 1
    if sbp >= 160:
        return(3) if treated == 1 else 2
    
def mage_pts(age):
    if age in range(45,49+1):
        return(3)
    if age in range(50,54+1):
        return(6)
    if age in range(55,59+1):
        return(8)
    if age in range(60,64+1):
        return(10)
    if age in range(65,69+1):
        return(11)
    if age in range(70,74+1):
        return(12)
    if age >= 75:
        return(13)

def mhdl_pts(hdl):
    if hdl >= 60:
        return(-1)
    if hdl in range(50,59+1):
        return(0)
    if hdl in range(40,49+1):
        return(1)
    if hdl < 40:
        return(2)

In [148]:
def get_frs_atp3(sex,age,hdl,totchol,sbp,curr_smq,treated):
    risk_score = 0
    if sex == 1:
        risk_score += mage_pts(age)
        risk_score += mhdl_pts(hdl)
        risk_score += mtchol_pts(totchol,age)
        risk_score += msbp_pts(sbp,treated)
        risk_score += msmoke_pts(curr_smq,age)
    if sex == 2:
        risk_score += fage_pts(age)
        risk_score += fhdl_pts(hdl)
        risk_score += ftchol_pts(totchol,age)
        risk_score += fsbp_pts(sbp,treated)
        risk_score += fsmoke_pts(curr_smq,age)
    return(risk_score)

def get_frs_atp3(x):
    risk_score = 0
    if x["SEX"] == 1:
        risk_score += mage_pts(x["AGEYR_AT_INT"])
        risk_score += mhdl_pts(x["lbdhdd"])
        risk_score += mtchol_pts(x["lbxtc"],x["AGEYR_AT_INT"])
        risk_score += msbp_pts(x["asbp"],x["treated"])
        risk_score += msmoke_pts(x["curr_smq"],x["AGEYR_AT_INT"])
    if x["SEX"] == 2:
        risk_score += fage_pts(x["AGEYR_AT_INT"])
        risk_score += fhdl_pts(x["lbdhdd"])
        risk_score += ftchol_pts(x["lbxtc"],x["AGEYR_AT_INT"])
        risk_score += fsbp_pts(x["asbp"],x["treated"])
        risk_score += fsmoke_pts(x["curr_smq"],x["AGEYR_AT_INT"])
    return(risk_score)
        
def f(x):    
    return x["treated"] + x["SEX"]

    Need to complete cases of all variable (fields) before running `get_frs` function

In [149]:
nhanesII=nhanesII.dropna(subset=["SEX","AGEYR_AT_INT","lbdhdd","lbxtc","asbp","curr_smq","treated"])
# count number of rows
nhanesII.SEX.count()

5218

In [150]:
nhanesII.apply(get_frs_atp3, axis=1)

0       15
1       18
2       15
3       19
4       17
        ..
5220    11
5221    10
5222    13
5223    18
5224    16
Length: 5218, dtype: int64

In [44]:
nhanesII["frs"] = nhanesII.apply(get_frs_atp3(nhanesII["SEX"],nhanesII["AGEYR_AT_INT"],nhanesII["lbdhdd"],nhanesII["lbxtc"],nhanesII["asbp"],nhanesII["curr_smq"],nhanesII["treated"]), axis=1)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().